In [1]:
import boto3
import pandas as pd
from io import StringIO
import time

In [2]:
AWS_ACCESS_KEY = "AKIA6BSGJ45QSZWVV4VGFQ"
AWS_SECRET_KEY = "WiqxntpfxdprSELuibgffcAMZrxKXW3IaRBlrkUuag"
AWS_REGION = "ap-southeast-1"
SCHEMA_NAME = "covid_dataset"
S3_STAGING_DIR = "s3://wangbao-covid-output/output/"
S3_BUCKET_NAME = "wangbao-covid-output"
S3_OUTPUT_DIRECTORY = "output"

In [3]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = AWS_REGION,                
)

In [4]:
Dict = {}
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            # This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_KEY,
        region_name = AWS_REGION,                
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

In [5]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

In [6]:
response

{'QueryExecutionId': 'd2498f67-fca1-44f1-80f6-75525134c819',
 'ResponseMetadata': {'RequestId': '9655b788-7605-4472-ba79-48076e0782a9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 29 May 2023 07:18:34 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9655b788-7605-4472-ba79-48076e0782a9'},
  'RetryAttempts': 0}}

In [7]:
enigma_jhud = download_and_load_query_results(athena_client, response)

In [8]:
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [9]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
nytimes_data_in_usa_us_county = download_and_load_query_results(athena_client, response)

In [10]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
nytimes_data_in_usa_us_states = download_and_load_query_results(athena_client, response)

In [11]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_covid_19_testing_data_states_daily = download_and_load_query_results(athena_client, response)

In [12]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_covid_19_testing_data_us_daily = download_and_load_query_results(athena_client, response)

In [13]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_covid_19_testing_data_us_total_latest = download_and_load_query_results(athena_client, response)

In [14]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)

In [15]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countrycode",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_countrycode = download_and_load_query_results(athena_client, response)

In [16]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countypopulation",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_countypopulation = download_and_load_query_results(athena_client, response)

In [17]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_state_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_state_abv = download_and_load_query_results(athena_client, response)

In [18]:
static_datasets_state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [19]:
new_header = static_datasets_state_abv.iloc[0]

In [20]:
new_header

col0           State
col1    Abbreviation
Name: 0, dtype: object

In [21]:
static_datasets_state_abv = static_datasets_state_abv[1:]
static_datasets_state_abv.columns = new_header

In [22]:
static_datasets_state_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [23]:
factCovid_1 = enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid_2 = rearc_covid_19_testing_data_states_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid = pd.merge(factCovid_1, factCovid_2, on='fips', how='inner')

In [24]:
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210307,101327.0,305972.0,147.0,NaN,NaN
1,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210306,101327.0,305972.0,147.0,NaN,NaN
2,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210305,101066.0,305972.0,136.0,NaN,NaN
3,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210304,100867.0,305972.0,171.0,NaN,NaN
4,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210303,100765.0,305972.0,169.0,NaN,NaN


In [25]:
factCovid.shape

(26418, 13)

In [26]:
# dimRegion_1 = enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
# dimRegion_2 = nytimes_data_in_usa_us_county[['fips','county','state']]
# dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')

In [27]:
#dimRegion

In [28]:
dimHospital = rearc_usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [29]:
dimDate = rearc_covid_19_testing_data_states_daily[['fips','date']]

In [30]:
dimDate.head()

,fips,date
0,2,20210307
1,1,20210307
2,5,20210307
3,60,20210307
4,4,20210307


In [31]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')

C:\Users\Admin\AppData\Local\Temp\ipykernel_4812\572748324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [32]:
dimDate.head()

,fips,date
0,2,2021-03-07
1,1,2021-03-07
2,5,2021-03-07
3,60,2021-03-07
4,4,2021-03-07


In [33]:
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek

C:\Users\Admin\AppData\Local\Temp\ipykernel_4812\935310350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\Admin\AppData\Local\Temp\ipykernel_4812\935310350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month
C:\Users\Admin\AppData\Local\Temp\ipykernel_4812\935310350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [34]:
dimDate.head()

,fips,date,year,month,day_of_week
0,2,2021-03-07,2021,3,6
1,1,2021-03-07,2021,3,6
2,5,2021-03-07,2021,3,6
3,60,2021-03-07,2021,3,6
4,4,2021-03-07,2021,3,6


In [35]:
bucket = 'wangbao-covid-de-project' # already created on S3

In [36]:
csv_buffer = StringIO()

In [37]:
csv_buffer

In [38]:
factCovid.to_csv(csv_buffer)

In [39]:
s3_resource = boto3.resource('s3')

In [40]:
s3_resource.Object(bucket, 'output/factCovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'JXXPWXGSH9XXXY8Y',
  'HostId': '62cy2mIcFIhvzyzRi6LE25+bvEfxrU2vckgPLEf4N4LMInt1ze9AxpL8jkX3uyrKCxGK1pHSi5n+G6w2YgWjEA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '62cy2mIcFIhvzyzRi6LE25+bvEfxrU2vckgPLEf4N4LMInt1ze9AxpL8jkX3uyrKCxGK1pHSi5n+G6w2YgWjEA==',
   'x-amz-request-id': 'JXXPWXGSH9XXXY8Y',
   'date': 'Mon, 29 May 2023 07:19:12 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"6372f86ce96858dcb69570e9500bc3c7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"6372f86ce96858dcb69570e9500bc3c7"',
 'ServerSideEncryption': 'AES256'}

In [41]:
csv_buffer = StringIO()
dimHospital.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimHospital.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'D1QN0F25JPKZ4XZF',
  'HostId': 'AIscoK6ylGkNLYO7RHxfwJC/pyNYek/vIHnFpjBpS4cSNzHscV5MpzaMhB6PLNGkFDLBbxzIuQrveNdjQhoxNg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'AIscoK6ylGkNLYO7RHxfwJC/pyNYek/vIHnFpjBpS4cSNzHscV5MpzaMhB6PLNGkFDLBbxzIuQrveNdjQhoxNg==',
   'x-amz-request-id': 'D1QN0F25JPKZ4XZF',
   'date': 'Mon, 29 May 2023 07:19:14 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a26c4e35d128fe6f64955ba9aac1d221"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"a26c4e35d128fe6f64955ba9aac1d221"',
 'ServerSideEncryption': 'AES256'}

In [42]:
csv_buffer = StringIO()
dimDate.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimDate.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'TY74MCJD7Y15WNAF',
  'HostId': 'kcmujO9rWVEfzNo71SG9frbNgPtBFAnoBODOTgPcYe6Gq+TIje+HCXxBP5Fj0siO/N8kxlNvNwQJSncFE2SACw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'kcmujO9rWVEfzNo71SG9frbNgPtBFAnoBODOTgPcYe6Gq+TIje+HCXxBP5Fj0siO/N8kxlNvNwQJSncFE2SACw==',
   'x-amz-request-id': 'TY74MCJD7Y15WNAF',
   'date': 'Mon, 29 May 2023 07:19:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d6eae31844ae905ae6342f60fffb5dc6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"d6eae31844ae905ae6342f60fffb5dc6"',
 'ServerSideEncryption': 'AES256'}

In [44]:
#dimRegion.to_csv('dimRegion.csv') # Then upload manually to s3

In [ ]:
# dimRegion.to_csv(csv_buffer)
# s3_resource.Object(bucket, 'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

In [45]:
dimDatesql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
print(''.join(dimDatesql))

CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [46]:
factCovidsql = pd.io.sql.get_schema(factCovid.reset_index(), 'factCovid')
print(''.join(factCovidsql))

CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [47]:
dimHospitalsql = pd.io.sql.get_schema(dimHospital.reset_index(), 'dimHospital')
print(''.join(dimHospitalsql))

CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


In [48]:
# dimRegionsql = pd.io.sql.get_schema(dimRegion.reset_index(), 'dimRegion')
# print(''.join(dimRegionsql))

In [49]:
#!pip install redshift-connector

In [50]:
import redshift_connector

In [51]:
conn = redshift_connector.connect(
    host='redshift-1.c5ekcidicr1l.ap-southeast-1.redshift.amazonaws.com',
    database='mydb',
    user='jabao',
    password='yourpass'
)

In [52]:
conn.autocommit = True

In [53]:
cursor = redshift_connector.Cursor = conn.cursor()

In [55]:
cursor.execute("""
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
""")

In [56]:
cursor.execute("""
copy dimDate from 's3://wangbao-covid-de-project/output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::964643580979:role/redshift-s3-access'
delimiter ','
region 'ap-southeast-1'
IGNOREHEADER 1
""")

In [57]:
cursor.execute("""
CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")

In [58]:
cursor.execute("""
copy dimHospital from 's3://wangbao-covid-de-project/output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::964643580979:role/redshift-s3-access'
delimiter ','
region 'ap-southeast-1'
IGNOREHEADER 1
""")

In [59]:
cursor.execute("""
CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

In [60]:
cursor.execute("""
copy factCovid from 's3://wangbao-covid-de-project/output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::964643580979:role/redshift-s3-access'
delimiter ','
region 'ap-southeast-1'
IGNOREHEADER 1
""")